<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/cpp_functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
functional programming in c++ をなぞり読み

https://www.manning.com/books/functional-programming-in-c-plus-plus#toc

In [ ]:
!sudo apt install librange-v3-dev

In [ ]:
# 最初の例
# 命令型の例
# main のないセルを実行するとエラーになる
# コメントアウトするか %%script false としておく
# Listing 1.1 Calculating the number of lines the imperative way
%%script false
vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

In [ ]:
# 動く形にして見よう
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

vector<int> count_lines_in_files(const vector<string>& files) {
    vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        ifstream in(file); 
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}

int main() {
    ofstream ofs01("./temp01.txt");
    ofs01 << "foo bar baz \nThis is a pen" << endl;
    ofs01.close(); // 2行
    ofstream ofs02("./temp02.txt");
    ofs02 << "日本語もどうか" << endl << endl;
    ofs02 << "これが最後" << endl << "ん" << endl; 
    ofs02.close(); // 4行

    vector<string> files = {"temp01.txt", "temp02.txt"};
    vector<int> lines;
    lines = count_lines_in_files(files);
    for(int i : lines) {
        cout << i << endl;
    }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# とりあえず動いた
# "\n" を数えているのでファイル末に改行がないと正しい数字にならない
# getline を使いたい

In [ ]:
# Listing 1.2 Using std::count to count newline characters
# count version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  ifstream in(filename);
  return count(istreambuf_iterator<char>(in), istreambuf_iterator<char>(), '\n');
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# count version は count を使っている事より、istreambuf_iterator を知らなかった
# cpprefjp による例
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int main()
{
  stringstream ss;
  ss << "1 2 3" << endl
     << "4 5 6";

  // 文字列の入力ストリームから順に文字を読み込むイテレータを用意
  istreambuf_iterator<char> it(ss);
  istreambuf_iterator<char> last;

  // イテレータを進めることにより、入力ストリームからデータを順に読み取る
  for_each(it, last, [](char c) { std::cout << c; });
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

1 2 3
4 5 6

In [ ]:
# ちょっとよくわかりません
# とりあえず getline version を作って、ファイル末の改行なしに対応するか実験しよう #=> 成功!!!!
# getline version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results;
  for (const auto& file : files) {
    results.push_back(count_lines(file));
  }
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# テキストは次に transform を使っている
# transform は他の言語では map に当たるアルゴリズムである
# map -> transform, reduce -> accumulate, filter -> copy_if らしい
# transform version
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<std::string>& files) {
    vector<int> results(files.size());
    // transform(files.cbegin(), files.cend(), results.begin(), count_lines);    
    transform(files.begin(), files.end(), results.begin(), count_lines); // cbegin と begin の違いは const かどうかでほとんど同じ
    return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# ranges version
# ranges は chapter 7 で説明するとのこと
# だがエラーなのでとりあえず %%script false にして先へ進むか
# 実験
# !sudo apt install librange-v3-dev
# include <range/v3/view/transform.hpp>
# ::ranges::view::transform
# としたら動いた
%%writefile temp.cpp
# include <bits/stdc++.h>
# include <range/v3/view/transform.hpp>

using namespace std;

int count_lines(const string& filename) {
  int count = 0;
  string myText;
  ifstream in(filename);
  while (getline (in, myText)) {
    count += 1;
  }
  return count;
}

vector<int> count_lines_in_files(const vector<string>& files) {
    return files | ::ranges::view::transform(count_lines);
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Writing temp.cpp


In [ ]:
!sudo apt install librange-v3-dev

In [ ]:
!g++ temp.cpp; ./a.out

2
4


In [ ]:
# 最終形は次のようになる、とのこと
# transform を 2回に分けている
%%script false
vector<int>
count_lines_in_files(const vector<string>& files) {
    return files | transform(open_file) | transform(count_lines);
}

In [ ]:
# Listing 2.1 Calculating the average score imperatively
# 平均を算出する 命令形
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double average_score(const vector<int>& scores) {
    int sum = 0;    
    for (int score : scores) {    
        sum += score;    
    }    
    return sum / (double)scores.size();    
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7.57143


In [ ]:
# Listing 2.2 Calculating the average score functionally
# 関数型
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double average_score(const vector<int>& scores) {
    return accumulate(scores.cbegin(), scores.cend(),0) / (double)scores.size();    
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ temp.cpp; ./a.out

7.57143


In [ ]:
# PARALLEL VERSIONS OF STANDARD ALGORITHMS
#  -std=c++17 にしても reduce , execution が使えないので %%script false
# 並行処理は一旦諦める
%%script false
%%writefile temp.cpp
# include <bits/stdc++.h>

using namespace std;

double average_score(const vector<int>& scores) {
    return reduce(execution::par, scores.cbegin(), scores.cend(),0) / (double) scores.length();
}

int main() {
    vector<int> vec = {9, 7, 10, 5, 8, 8, 6};
    double average;
    average = average_score(vec);
    cout << average << endl;
}

Overwriting temp.cpp


In [ ]:
!g++ -std=c++17 temp.cpp

In [ ]:
# c++17 の any は -std=c++17 して include  すれば使える
# reduce や execution は同じようにしてもエラーになる
%%writefile temp.cpp
# include <bits/stdc++.h>
# include <any>
using namespace std;

int main(){
    auto s = any(123);
    cout << any_cast<int>(s) << endl; // "123"が標準出力
}

Overwriting temp.cpp


In [ ]:
!g++ -std=c++17 temp.cpp; ./a.out

123


In [ ]:
# Listing 2.3 Calculating the product of all scores
# accumulate の使い方 関数が渡せる fold や reduce みたい
# この例では multiplies<int>() というのを渡している
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

double scores_product(const vector<int>& scores) {
  return accumulate(scores.cbegin(), scores.cend(), 1, multiplies<int>());
}

int main() {
  vector<int> vec = { 2,3,4 };
  cout << scores_product(vec) << endl;  //=> 24
}

Overwriting temp.cpp


In [ ]:
!g++  temp.cpp; ./a.out

24


In [66]:
# folding
# Counting newline characters with accumulate
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

int f(int previous_count, char c) {
  return (c != '\n') ? previous_count : previous_count + 1;
}

int count_lines(const string& s) {
  return accumulate(s.cbegin(), s.cend(), 0, f);
}

int count_lines1(const string& filename) {
  ifstream in(filename);
  stringstream myStream;
  myStream << in.rdbuf();
  return count_lines(myStream.str());
}

vector<int> count_lines_in_files(const vector<string>& files) {
  vector<int> results(files.size());
  transform(files.begin(), files.end(), results.begin(), count_lines1);
  return results;
}

int main() {
  ofstream ofs01("./temp01.txt");
  ofs01 << "foo bar baz \nThis is a pen" << endl;
  ofs01.close(); // 2行
  ofstream ofs02("./temp02.txt");
  ofs02 << "日本語もどうか" << endl << endl;
  // ofs02 << "これが最後" << endl << "ん" << endl; 
  ofs02 << "これが最後" << endl << "ん"; 
  ofs02.close(); // 4行

  vector<string> files = { "temp01.txt", "temp02.txt" };
  vector<int> lines;
  lines = count_lines_in_files(files);
  for (int i : lines) {
    cout << i << endl;
  }
}

Overwriting temp.cpp


In [67]:
!g++  temp.cpp; ./a.out

2
3


In [68]:
# 2.2.3 String trimming
%%script false
string trim_left(string s) {
    s.erase(s.begin(), find_if(s.begin(), s.end(), is_not_space));
    return s;
}

In [ ]:
%%script false
string trim_right(string s) {
    s.erase(find_if(s.rbegin(), s.rend(), is_not_space).base(), s.end());
    return s;
}

In [ ]:
%%script false
string trim(string s) {
    return trim_left(trim_right(move(s)));
}

In [81]:
# 動くプログラムになるか
# folding
# Counting newline characters with accumulate
%%writefile temp.cpp
# include <bits/stdc++.h>
using namespace std;

bool is_not_space(char c) {
    return (c != ' ');
} 

string trim_left(string s) {
    s.erase(s.begin(), find_if(s.begin(), s.end(), is_not_space));
    return s;
}

string trim_right(string s) {
    s.erase(find_if(s.rbegin(), s.rend(), is_not_space).base(), s.end());
    return s;
}

string trim(string s) {
    return trim_left(trim_right(move(s)));
}

int main() {
    string str = "  this is a pen  ";
    cout << "(begin)" << str << "(end)" << endl;
    cout << "(begin)" << trim(str) << "(end)" << endl;
}

Overwriting temp.cpp


In [82]:
!g++  temp.cpp; ./a.out

(begin)  this is a pen  (end)
(begin)this is a pen(end)


In [ ]:
# 2.2.4 Partitioning collections based on a predicate
# Listing 2.5 Females first

# partition(people.begin(), people.end(),is_female);

# Listing 2.6 Moving selected items to a specific point

# stable_partition(first, destination, is_not_selected);
# stable_partition(destination, last,  is_selected);

In [ ]:
# 2.2.5 Filtering and transforming
# bool is_female(const person_t& person);
# bool is_not_female(const person_t& person);
# string name(const person_t& person);

In [ ]:
# Listing 2.7 Filtering items by removing undesired ones
# people.erase(remove_if(people.begin(), people.end(),is_not_female),people.end());

In [ ]:
# Listing 2.9 Getting the names
# vector<string> names(females.size());  
# transform(females.cbegin(), females.cend(),names.begin(),name);

In [ ]:
# 2.3 Composability problems of STL algorithms
# vector<string> names;
# for (const auto& person : people) {
#     if (is_female(person)) {
#         names.push_back(name(person));
#     }
# }

In [ ]:
# filter    : (collection<T>, (T → bool)) → collection<T>
# transform : (collection<T>, (T → T2)) → collection<T2>

# transform(filter(people, is_female), name)

In [ ]:
# OutputIt copy_if(InputIt first, InputIt last,
#                  OutputIt destination_begin,
#                  UnaryPredicate pred);
# 
# OutputIt transform(InputIt first, InputIt last,
#                    OutputIt destination_begin,
#                    UnaryOperation transformation);

In [ ]:
# Listing 2.10 Females first
# vector<person_t> separated(people.size());
# 
# const auto last = copy_if(    
#         people.cbegin(), people.cend(),
#         separated.begin(),
#         is_female);
# 
# copy_if(
#         people.cbegin(), people.cend(),
#         last,    
#         is_not_female);

In [ ]:
# 2.4 Writing your own higher-order functions
# 2.4.1 Receiving functions as arguments
# template <typename FilterFunction>
# vector<string> names_for(
#         const vector<person_t>& people,
#         FilterFunction filter)

In [ ]:
# 2.4.2 Implementing with loops
# Listing 2.11 Implementing the function by using a handwritten loop
# template <typename FilterFunction>
# vector<string> names_for(
#         const vector<person_t>& people,
#         FilterFunction filter)
# {
#     vector<string> result;

#     for (const person_t& person : people) {
#         if (filter(person)) {
#             result.push_back(name(person));
#         }
#     }

#     return result;
# }

In [ ]:
# 2.4.3 Recursion and tail-call optimization
# Listing 2.12 Naive recursive implementation
# template <typename FilterFunction>
# vector<string> names_for(
#         const vector<person_t>& people,
#         FilterFunction filter)
# {
#     if (people.empty()) {
#         return {};    

#     } else {
#         const auto head = people.front();
#         const auto processed_tail = names_for(    
#                 tail(people),
#                 filter);

#         if (filter(head)) {    
#             return prepend(name(head), processed_tail);    
#         } else {    
#             return processed_tail;    
#         }
#     }
# }

In [ ]:
# Listing 2.13 Recursive implementation
# template <typename FilterFunction, typename Iterator>
# vector<string> names_for(
#         Iterator people_begin,
#         Iterator people_end,
#         FilterFunction filter)
# {
#         …
#         const auto processed_tail = names_for(
#                 people_begin + 1,
#                 people_end,
#                 filter);
#         …
# }

In [ ]:
# Listing 2.14 Tail-recursive implementation
# template <typename FilterFunction, typename Iterator>
# vector<string> names_for_helper(
#         Iterator people_begin,
#         Iterator people_end,
#         FilterFunction filter,
#         vector<string> previously_collected)
# {
#     if (people_begin == people_end) {
#         return previously_collected;

#     } else {
#         const auto head = *people_begin;

#         if (filter(head)) {
#             previously_collected.push_back(name(head));
#         }

#         return names_for_helper(
#                 people_begin + 1,
#                 people_end,
#                 filter,
#                 move(previously_collected));
#     }
# }

In [ ]:
# Listing 2.15 Calling the helper function
# template <typename FilterFunction, typename Iterator>
# vector<string> names_for(
#         Iterator people_begin,
#         Iterator people_end,
#         FilterFunction filter)
# {
#     return names_for_helper(people_begin,
#                             people_end,
#                             filter,
#                             {});
# }

In [ ]:
# 2.4.4 Implementing using folds
# Listing 2.16 Implementation using folding
# vector<string> append_name_if(
#         vector<string> previously_collected,
#         const person_t& person)
# {
#     if (filter(person)) {
#         previously_collected.push_back(name(person));
#     }
#     return previously_collected;
# }

# …

# return accumulate(
#         people.cbegin(),
#         people.cend(),
#         vector<string>{},
#         append_name_if);

In [ ]:
# 3 Function objects
# 3.1 Functions and function objects
# int max(int arg1, int arg2) { … }
# auto max(int arg1, int arg2) -> int { … }

In [ ]:
# 3.1.1 Automatic return type deduction
# int answer = 42;
# auto ask1() { return answer; }    
# const auto& ask2() { return answer; }

In [ ]:
# auto ask(bool flag) {
#     if (flag) return 42;
#     else      return string("42");    
# }

In [ ]:
# auto factorial(int n)
# {
#     if (n == 0) {
#         return 1;    
#     } else {
#         return factorial(n - 1) * n;    
#     }
# }

In [ ]:
# decltype(auto) ask() { return answer; }    
# decltype(auto) ask() { return (answer); }    
# decltype(auto) ask() { return 42 + answer; }

In [ ]:
# template <typename Object, typename Function>
# decltype(auto) call_on_object(Object&& object, Function function)
# {
#     return function(forward<Object>(object));
# }

In [ ]:
# Perfect forwarding for arguments
# template <typename Object, typename Function>
# decltype(auto) call_on_object(Object object,
#                               Function function)
# {
#     return function(object);
# }

In [ ]:
# template <typename T>
# void f(T&& fwd, int&& value) { … }

In [ ]:
# 3.1.2 Function pointers
# int ask() { return 42; }

# typedef decltype(ask)* function_ptr;

# class convertible_to_function_ptr {
# public:
#     operator function_ptr() const    
#     {    
#         return ask;    
#     }    
# };

# int main(int argc, char* argv[])
# {
#     auto ask_ptr = &ask;    
#     :cout << ask_ptr() << '\n';    

#     auto& ask_ref = ask;    
#     :cout << ask_ref() << '\n';    

#     convertible_to_function_ptr ask_wrapper;    
#     :cout << ask_wrapper() << '\n';    
# }

In [ ]:
# 3.1.3 Call operator overloading
# class function_object {
# public:
#     return_type operator()(arguments) const
#     {
#         …
#     }
# };

In [ ]:
# bool older_than_42(const person_t& person) {
#     return person.age > 42;
# }
# count_if(persons.cbegin(), persons.cend(),
#               older_than_42);

In [ ]:
# class older_than {
# public:
#     older_than(int limit)
#         : m_limit(limit)
#     {
#     }

#     bool operator()(const person_t& person) const
#     {
#         return person.age() > m_limit;
#     }

# private:
#     int m_limit;
# };

In [ ]:
# older_than older_than_42(42);
# older_than older_than_14(14);

# if (older_than_42(person)) {
#     std::cout << person.name() << " is more than 42 years old\n";
# } else if (older_than_14(person)) {
#     std::cout << person.name() << " is more than 14 years old\n";
# } else {
#     std::cout << person.name() << " is 14 years old, or younger\n";
# }

In [ ]:
# count_if(persons.cbegin(), persons.cend(),
#               older_than(42));
# count_if(persons.cbegin(), persons.cend(),
#               older_than(16));

In [ ]:
# 3.1.4 Creating generic function objects
# template <typename T>
# class older_than {
# public:
#     older_than(int limit)
#         : m_limit(limit)
#     {
#     }

#     bool operator()(const T& object) const
#     {
#         return object.age()> m_limit;
#     }

# private:
#     int m_limit;
# };

In [ ]:
# count_if(persons.cbegin(), persons.cend(),
#               older_than<person_t>(42));
# count_if(cars.cbegin(), cars.cend(),
#               older_than<car_t>(5));
# count_if(projects.cbegin(), projects.cend(),
#               older_than<project_t>(2));

In [ ]:
# Listing 3.1 Creating a function object with a generic call operator
# class older_than {
# public:
#     older_than(int limit)
#         : m_limit(limit)
#     {
#     }

#     template <typename T>
#     bool operator()(T&& object) const
#     {
#         return std::forward<T>(object).age()> m_limit;    
#     }

# private:
#     int m_limit;
# };

In [ ]:
# Listing 3.2 Using a function object with a generic call operator
# older_than predicate(5);

# count_if(persons.cbegin(), persons.cend(), predicate);
# count_if(cars.cbegin(), cars.cend(), predicate);
# count_if(projects.cbegin(), projects.cend(), predicate);
# copy

In [ ]:
# 3.2 Lambdas and closures
# std::copy_if(people.cbegin(), people.cend(),
#              std::back_inserter(females),
#              [](const person_t& person) {
#                  return person.gender() == person_t::female;
#              }
#         );

In [ ]:
# 3.2.2 Under the hood of lambdas
# class company_t {
# public:
#     string team_name_for(const person_t& employee) const;

#     int count_team_members(const string& team_name) const;

# private:
#     vector<person_t> m_employees;
#      …
# };

In [ ]:
# Listing 3.3 Counting the number of employees in a given team
# int company_t::count_team_members(
#         const string& team_name) const
# {
#     return count_if(
#             m_employees.cbegin(), m_employees.cend(),
#             [this, &team_name]    
#                 (const person_t& employee)    
#             {
#                 return team_name_for(employee) ==
#                        team_name;
#             }
#         );
# }

In [ ]:
# Listing 3.4 Lambda converted to a class
# class lambda_implementation {
# public:
#     lambda_implementation(
#             const company_t* _this,
#             const string& team_name)
#         : m_this(_this)
#         , m_team_name(team_name)
#     {
#     }

#     bool operator()(const person_t& employee) const
#     {
#         return m_this->team_name_for(employee) == m_team_name;
#     }

# private:
#     const company_t* m_this;
#     const string& m_team_name;
# };

In [ ]:
# Listing 3.5 Creating a mutable lambda
# int count = 0;
# vector<string> words{"An", "ancient", "pond"};

# for_each(words.cbegin(), words.cend(),
#         [count]    
#         (const string& word)
#         mutable    
#         {
#             if (isupper(word[0])) {
#                 cout << word
#                           << " " << count <<endl;
#                 count++;
#             }
#         }
#     );

In [ ]:
# 3.2.3 Creating arbitrary member variables in lambdas
# Listing 3.6 Error when trying to capture a move-only type
# unique_ptr<session_t> session = create_session();

# auto request = server.request("GET /", session->id());

# request.on_completed(
#             [session]    
#             (response_t response)
#             {
#                 cout << "Got response: " << response
#                           << " for session: " << session;
#             }
#         );

In [ ]:
# Listing 3.7 Generalized lambda captures
# request.on_completed(
#            [ session = move(session),    
#              time = current_time()    
#            ]
#            (response_t response)
#            {
#                 cout
#                     << "Got response: " << response
#                     << " for session: " << session
#                     << " the request took: "
#                     << (current_time() - time)
#                     << "milliseconds";
#            }
#         );

In [ ]:
# 3.2.4 Generic lambdas
# Listing 3.8 Generic lambda that accepts objects with age()
# auto predicate = [limit = 42](auto&& object) {    
#     return object.age() > limit;    
# };    

# std::count_if(persons.cbegin(), persons.cend(),
#               predicate);
# std::count_if(cars.cbegin(), cars.cend(),
#               predicate);
# std::count_if(projects.cbegin(), projects.cend(),
#               predicate);

In [ ]:
# MORE-GENERIC LAMBDAS IN C++20
# [] (auto first, decltype(first) second) { … }

# [] <typename T> (T first, T second) { … }

In [ ]:
# 3.3 Writing function objects that are even terser than lambdas
# ok_responses = filter(responses,
#         [](const response_t& response) {
#             return !response.error();
#         });
# failed_responses = filter(responses,
#         [](const response_t& response) {
#             return response.error();
#         });

In [ ]:
# ok_responses     = filter(responses, _.error() == false);
# failed_responses = filter(responses, _.error());

In [ ]:
# ok_responses     = filter(responses, not_error);
#     / or          filter(responses, !error);
#     / or          filter(responses, error == false);

# failed_responses = filter(responses, error);
#     / or          filter(responses, error == true);
#     / or even     filter(responses, not_error == false);

In [ ]:
# Listing 3.9 Basic implementation of a predicate to test for errors
# class error_test_t {
# public:
#     error_test_t(bool error = true)
#         : m_error(error)
#     {
#     }

#     template <typename T>
#     bool operator()(T&& value) const
#     {
#         return m_error ==    
#                 (bool)std::forward<T>(value).error();    
#     }

# private:
#     bool m_error;
# };

# error_test_t error(true);
# error_test_t not_error(false);

In [ ]:
# Listing 3.10 Defining convenient operators for the predicate function object
# class error_test_t {
# public:
#     …

#     error_test_t operator==(bool test) const
#     {
#         return error_test_t(    
#                 test ? m_error : !m_error    
#             );    
#     }

#     error_test_t operator!() const
#     {
#         return error_test_t(!m_error);    
#     }

#     …
# };

In [ ]:
# 3.3.1 Operator function objects in STL
# vector<int> numbers{1, 2, 3, 4};

# product = accumulate(numbers.cbegin(), numbers.cend(), 1,
#                           multiplies<int>());

# / product is 24

In [ ]:
# vector<int> numbers{5, 21, 13, 42};

# sort(numbers.begin(), numbers.end(), greater<int>());

# / numbers now contain {42, 21, 13, 5}

In [2]:
# Table 3.1 Operator wrappers available in the standard library
%%html
<table border="1">
<thead>
<tr><th>Group</th><th>Wrapper name</th><th>Operation</th></tr>
</thead>
<tbody>
<tr><td>Arithmetic operators</td><td>plus</td><td>arg_1 + arg_2</td></tr>
<tr><td></td><td>minus</td><td>arg_1 - arg_2</td></tr>
<tr><td></td><td>multiplies</td><td>arg_1 * arg_2</td></tr>
<tr><td>Arithmetic operators (continued)</td><td>divides</td><td>arg_1 / arg_2</td></tr>
<tr><td></td><td>modulus</td><td>arg_1 % arg_2</td></tr>
<tr><td></td><td>negates</td><td>- arg_1 (a unary function)</td></tr>
<tr><td>Comparison operators</td><td>equal_to</td><td>arg_1 == arg_2</td></tr>
<tr><td></td><td>not_equal_to</td><td>arg_1 != arg_2</td></tr>
<tr><td></td><td>greater</td><td>arg_1 > arg_2</td></tr>
<tr><td></td><td>less</td><td>arg_1 < arg_2</td></tr>
<tr><td></td><td>greater_equal</td><td>arg_1 >= arg_2</td></tr>
<tr><td></td><td>less_equal</td><td>arg_1 <= arg_2</td></tr>
<tr><td>Logical operators</td><td>logical_and</td><td>arg_1 && arg_2</td></tr>
<tr><td></td><td>logical_or</td><td>arg_1 || arg_2</td></tr>
<tr><td></td><td>logical_not</td><td>!arg_1 (a unary function)</td></tr>
<tr><td>Bitwise operators</td><td>bit_and</td><td>arg_1 & arg_2</td></tr>
<tr><td></td><td>bit_or</td><td>arg_1 | arg_2</td></tr>
<tr><td></td><td>bit_xor</td><td>arg_1 ^ arg_2</td></tr>
</tbody>
</table>

Group,Wrapper name,Operation
Arithmetic operators,plus,arg_1 + arg_2
,minus,arg_1 - arg_2
,multiplies,arg_1 * arg_2
Arithmetic operators (continued),divides,arg_1 / arg_2
,modulus,arg_1 % arg_2
,negates,- arg_1 (a unary function)
Comparison operators,equal_to,arg_1 == arg_2
,not_equal_to,arg_1 != arg_2
,greater,arg_1 > arg_2
,less,arg_1 < arg_2


In [ ]:
# THE DIAMOND ALTERNATIVE
# sort(numbers.begin(), numbers.end(),
#             std::greater<>());

In [ ]:
# 3.3.2 Operator function objects in other libraries
# BOOST LIBRARIES
# using namespace boost::phoenix::arg_names;

# vector<int> numbers{21, 5, 62, 42, 53};

# partition(numbers.begin(), numbers.end(),
#                arg1 <= 42);

# / numbers now contain {21, 5, 42,   62, 53}
# /                        <= 42       > 42

In [ ]:
# accumulate(numbers.cbegin(), numbers.cend(), 0,
#                 arg1 + arg2 * arg2 / 2);

In [ ]:
# product = accumulate(numbers.cbegin(), numbers.cend(), 1,
#                           arg1 * arg2);

# sort(numbers.begin(), numbers.end(), arg1 > arg2);

In [ ]:
# 3.4 Wrapping function objects with function

# function<float(float, float)> test_function;    

# test_function = fmaxf;    

# test_function = multiplies<float>();    

# test_function = multiplies<>();    

# test_function = [x](float a, float b) { return a * x + b; };    

# test_function = [x](auto a, auto b) { return a * x + b; };    

# test_function = (arg1 + arg2) / 2;    

# test_function = [](string s) { return s.empty(); } / ERROR!

In [ ]:
# string str{"A small pond"};
# function<bool(std::string)> f;

# f = &std::string::empty;

# cout << f(str);

In [ ]:
4 Creating new functions from the old ones


In [ ]:
!g++  temp.cpp; ./a.out

# いまここ